<a href="https://colab.research.google.com/github/rubyvanrooyen/ARIWS-Cookbook/blob/main/utils/Galactic_coordinates_to_RA_DEC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from astropy.coordinates import SkyCoord
from astropy import units as u
from astropy.coordinates import Galactic, ICRS

from ipywidgets import widgets
from IPython.display import display
from ipywidgets import interact

from numpy import recarray
import os

Position coordinates for galactic targets are converted to celestial target coordinates.    
Calibrator targets can then be added and elevation plots generated of all targets

Input coordinates assumes the (`HMS`, `DMS`) string format for both galactic $(l, b)$ and celestial $(\alpha, \delta)$ coordinates.

In [2]:
class ObsTargets(object):
    def __init__(self):
        self.dtype = {'names': ('name', 'gal', 'radec'),
                      'formats': ('S12', SkyCoord, SkyCoord),
                      }
        self.targets = recarray((0,), dtype=self.dtype)
        
    def add_target(self, name, gal_coord, radec_coord):
        self.targets.resize(self.targets.size+1)
        self.targets[-1] = (name, gal_coord, radec_coord)
    
    def clear_targets(self):
        self.targets = recarray((0,), dtype=self.dtype)
    
    def add_gal_coordinate(self, name, glon_str, glat_str):
        gal_coord = SkyCoord(l=glon_str, b=glat_str, frame=Galactic)
        radec_coord = gal_coord.transform_to(ICRS)
        self.add_target(name, gal_coord, radec_coord)
    
    def add_radec_coordinate(self, name, ra_str, dec_str):
        radec_coord = SkyCoord(ra=ra_str, dec=dec_str,
                               unit=(u.hourangle, u.deg),
                               frame=ICRS)
        gal_coord = radec_coord.transform_to(Galactic)
        self.add_target(name, gal_coord, radec_coord)
    
    def show_gal_coordinates(self):
        str_list = ''
        for target in self.targets:
            str_list +="name={}, gal={}\n".format(target.name,
                                                  target.gal.to_string('hmsdms'))
        return str_list
    
    def show_radec_coordinates(self):
        str_list = ''
        for target in self.targets:
            ra_str = target.radec.ra.to_string(unit=u.hourangle,
                                               sep=':',
                                               precision=4,
                                               pad=True)
            dec_str = target.radec.dec.to_string(sep=':',
                                                 precision=4,
                                                 alwayssign=True,
                                                 pad=True)
            str_list +="Name={}, radec={} {}\n".format(target.name,
                                                       str(ra_str),
                                                       str(dec_str))
        return str_list
    
    def write_file(self, filename, outstr, verbose=False):
        with open(filename, 'w') as fout:
            fout.write(outstr)
        print('File saved as: {}'.format(os.path.join(os.getcwd(),filename)))
        if verbose:
            print(outstr)
    
    def write_gal_csv(self, filename='gal_coords.csv'):
        outstr = ''
        for target in self.targets:
            outstr += '{}, gal target, {:.3f}, {:.3f}\n'.format(target.name,
                                                                target.gal.l.degree,
                                                                target.gal.b.degree)
        self.write_file(filename, outstr)
        
    def write_radec_csv(self, filename='radec_coords.csv', verbose=False):
        outstr = ''
        for target in self.targets:
            ra_str = target.radec.ra.to_string(unit=u.hourangle, sep=':', precision=3,pad=True)
            dec_str = target.radec.dec.to_string(sep=':',precision=3, alwayssign=True, pad=True)
            outstr += '{}, radec target, {}, {}\n'.format(target.name,
                                                          ra_str,
                                                          dec_str)
        self.write_file(filename, outstr, verbose=verbose)

In [3]:
class list_inputs():
    def __init__(self,
                 gname='', 
                 glon='',  
                 glat='',  
                 cname='', 
                 ra='',  
                 dec='',  
                 prefill=False,
                ):
        
        if prefill:
            gname, glon, glat = 'Test', '1h12m43.2s', '+1d12m43s'
            cname, ra, dec = 'Test', '00h42m44.2992s', '+41d16m09.012s'
                
        self.gname = self._add_text_(description='Name', value=gname)
        # The Galactic longitude for this object
        self.glon = self._add_text_(description='Lon (l)', value=glon)
        # The Galactic latitude for this object
        self.glat = self._add_text_(description='Lat (b)', value=glat)
        # Create explicit add button
        self.add_gbutton = widgets.Button(description='Add target', layout={'width':'auto'})
        
        self.cname = self._add_text_(description='Name', value=cname)
        # The right ascension for this object
        self.ra = self._add_text_(description='RA', value=ra)
        # The declination for this object
        self.dec = self._add_text_(description='Dec', value=dec)
        # Create explicit add button
        self.add_cbutton = widgets.Button(description='Add target', layout={'width':'auto'})
        
        # Create explicit clear button
        gal_header = widgets.Button(description='Enter galactic target coordinates',
                                    disabled=True, button_style='', tooltip='', icon='',
                                    layout={'width':'608px'})        
        header_0_0 = self._add_header_(description='T11R00C02')
        header_0_1 = self._add_header_(description='21h51m30.37s')        
        header_0_2 = self._add_header_(description='+00d59m15.56s')        
 
        radec_header = widgets.Button(description='Enter celestial target coordinates',
                                      disabled=True, button_style='', tooltip='', icon='',
                                      layout={'width':'608px'})        
        header_1_0 = self._add_header_(description='J1939-6342')
        header_1_1 = self._add_header_(description='19h39m25.0264s')        
        header_1_2 = self._add_header_(description='-63d42m45.624s')        
 
        headers_0 = widgets.HBox([header_0_0, header_0_1, header_0_2])
        galactic_target = widgets.HBox([self.gname, self.glon, self.glat, self.add_gbutton])
        headers_1 = widgets.HBox([header_1_0, header_1_1, header_1_2])
        celestial_target = widgets.HBox([self.cname, self.ra, self.dec, self.add_cbutton])
        self.ui = widgets.VBox([gal_header, headers_0, galactic_target,
                                radec_header, headers_1, celestial_target])

    def _add_header_(self, description):
        header_layout = {'width':'200px', 'min_width':'200px',
                         'height':'28px', 'min_height':'28px'}
        return widgets.Button(description=description,
                              disabled=True,
                              button_style='',
                              tooltip='',
                              icon='',
                              layout=header_layout)

    def _add_text_(self, description, value=''):
        text_style = {'description_width': 'initial'}
        text_layout = {'width':'200px', 'min_width':'200px'}
        return widgets.Text(description=description,
                            value=value,
                            layout=text_layout,
                            style=text_style)

In [4]:
class list_output():
    def __init__(self):
        row_style = {'description_width': 'initial'}
        row_layout = {'width':'400px', 'min_width':'400px', 'height':'200px', 'min_height':'200px'}
        self.galactice_coordinates = widgets.Textarea(layout=row_layout, style=row_style)
        self.celestial_coordinates = widgets.Textarea(layout=row_layout, style=row_style)
        # Create explicit clear button
        self.clear_button = widgets.Button(description='Clear lists', layout={'width':'803px'})
        output_coordinates = widgets.HBox([self.galactice_coordinates,
                                           self.celestial_coordinates])
        
        header_layout = {'width':'400px', 'min_width':'400px', 'height':'28px', 'min_height':'28px'}
        header_0 = widgets.Button(description='Galactic target list',
                                  disabled=True,
                                  button_style='',
                                  tooltip='',
                                  icon='',
                                  layout=header_layout)
        header_1 = widgets.Button(description='Celestial target list',
                                  disabled=True,
                                  button_style='',
                                  tooltip='',
                                  icon='',
                                  layout=header_layout)
        headers = widgets.HBox([header_0, header_1])
        
        self.save_gal = widgets.Button(description='Save as CSV',
                                       layout=header_layout)
        self.save_radec = widgets.Button(description='Save as CSV',
                                         layout=header_layout)
        savebuttons = widgets.HBox([self.save_gal, self.save_radec])
        
        self.ui = widgets.VBox([self.clear_button, savebuttons, headers, output_coordinates])

In [5]:
def decode(wdgt_text):
    if isinstance(wdgt_text, str):
        text = wdgt_text
    else:
        text = wdgt_text.decode(encoding)
    return text

In [6]:
obs_targets = ObsTargets()
inputs = list_inputs(prefill=True)
outputs = list_output()

# Define function to bind value of the input to the output variable
def update_output(obs_targets):
    outputs.galactice_coordinates.value = decode(obs_targets.show_gal_coordinates())
    outputs.celestial_coordinates.value = decode(obs_targets.show_radec_coordinates())

def add_gal_target(sender):
    obs_targets.add_gal_coordinate(str(inputs.gname.value),
                                   str(inputs.glon.value),
                                   str(inputs.glat.value))
    update_output(obs_targets)
    
def add_radec_target(sender):
    obs_targets.add_radec_coordinate(str(inputs.cname.value),
                                     str(inputs.ra.value),
                                     str(inputs.dec.value))
    update_output(obs_targets)
    
def clear_lists(wdgt):
    obs_targets.clear_targets()
    outputs.galactice_coordinates.value = decode(obs_targets.show_gal_coordinates())
    outputs.celestial_coordinates.value = decode(obs_targets.show_radec_coordinates())

def write_gal_targets(sender):
    obs_targets.write_gal_csv()

def write_radec_targets(sender):
    obs_targets.write_radec_csv(verbose=True)

# functionality
print("Enter your target coordinates (either in galactic or equatorial coordinates) below")
print("And press 'Add target' button to display the results")
print("The results for each target added will be append the lists displayed")
display(inputs.ui)
display(outputs.ui)
inputs.add_gbutton.on_click(add_gal_target)
inputs.add_cbutton.on_click(add_radec_target)
outputs.clear_button.on_click(clear_lists)
outputs.save_gal.on_click(write_gal_targets)
outputs.save_radec.on_click(write_radec_targets)

Enter your target coordinates (either in galactic or equatorial coordinates) below
And press 'Add target' button to display the results
The results for each target added will be append the lists displayed
